In [ ]:
import numpy as np
from scipy.sparse import csr_matrix
import time
import matplotlib.pyplot as plt

# Paste your `im2col_matrix_sparse` from the previous exercise

In [ ]:
# im2col
def im2col_matrix_sparse(Xin, K, S=1):
    N, Cin, Hin, Win = Xin.shape
    CHW = Cin * Hin * Win
    Hout = (Hin - K)//S + 1
    Wout = (Win - K)//S + 1
    P = Hout * Wout  # Total number of patches per image
    patch_size = Cin * K * K # Size of each flattened patch

    data = [1 for _ in range(P*patch_size)]
    row_indices = []
    col_indices = list(range(P*patch_size))

    patch_idx = 0
    for hout in range(Hout):
        for wout in range(Wout):
            for cin in range(Cin):
                for hker in range(K):
                    for wker in range(K):
                        input_index = cin * Hin * Win + hout * S * Win + wout * S + hker * Win + wker
                        row_indices.append(input_index)
            patch_idx += 1

    im2col_mat_sparse = csr_matrix((data, (row_indices, col_indices)), shape=(CHW, P * patch_size))
    return im2col_mat_sparse




# # im2col for dense matrix
# def im2col_matrix_dense(Xin, K, S=1):
#     N, Cin, Hin, Win = Xin.shape
#     Hout, Wout = (Hin - K)//S + 1, (Win - K)//S + 1
#     P = Hout * Wout  # Total number of patches per image
#     patch_size = Cin * K * K # Size of each flattened patch
#     im2col_mat_dense = np.zeros((Cin * Hin * Win, P * patch_size))
#     patch_idx = 0
#     output_index = 0
#     for hout in range(Hout):
#         for wout in range(Wout):
#             for cin in range(Cin):
#                 for hker in range(K):
#                     for wker in range(K):
#                         input_index = cin * Hin * Win + hout * S * Win + wout * S + hker * Win + wker
#                         im2col_mat_dense[input_index, output_index] = 1
#                         output_index += 1
#             patch_idx += 1
#     return im2col_mat_dense


In [ ]:
# broadcasting

# Broadcasted shape
def broadcasted_shape(shape_X, shape_Y):

    max_len = max(len(shape_X), len(shape_Y))
    min_len = min(len(shape_X), len(shape_Y))
    swapped = False

    if len(shape_X) < max_len:
        swapped = True
        L, S = shape_Y, shape_X # L = long, S = short
    else:
        L, S = shape_X, shape_Y

    L_rev = L[::-1]
    S_rev = S[::-1]

    result_shape = []
    axes_L_expanded = []
    axes_S_expanded = []

    for i in range(min_len):
        dim_L = L_rev[i]
        dim_S = S_rev[i]
        if dim_L == 1 and dim_S != 1:
            axes_L_expanded.append(max_len  -1- i)
        elif dim_L != 1 and dim_S == 1:
            axes_S_expanded.append(max_len  -1- i)
        if dim_L == 1 or dim_S == 1 or dim_L == dim_S:
            result_shape.append(max(dim_L,dim_S))
        else:
            raise ValueError(f"Shapes {shape_X} and {shape_Y} not broadcastable")


    result_shape += L_rev[(min_len):]

    result_shape = tuple(result_shape[::-1])
    axes_L_expanded = tuple(axes_L_expanded[::-1])
    axes_S_expanded = tuple(axes_S_expanded[::-1])

    if swapped:
        return result_shape, axes_S_expanded, max_len - min_len,  axes_L_expanded, 0
    else:
        return result_shape,  axes_L_expanded, 0, axes_S_expanded, max_len - min_len

def unbroadcast(arr, ax, pad):
    return np.sum(np.sum(arr, axis = ax, keepdims = True), axis = tuple(range(pad)))

In [ ]:
class ag: # AutoGrad

    #################
    # ENTRYWISE OPS #
    #################
    def log(input):
        output = ag.Tensor(np.log(input.value), inputs=[input], op="log")
        def _backward():
            input.grad += output.grad / input.value
            return None
        output._backward = _backward
        return output

    def exp(input):

        output = ag.Tensor(np.exp(input.value), inputs=[input], op="exp")

        def _backward():
            input.grad += output.grad * output.value
            return None

        output._backward = _backward
        return output

    def relu(input):
        output = ag.Tensor(np.maximum(0, input.value), inputs=[input], op="relu")

        def _backward():
            input.grad += (input.value > 0)*output.grad
            return None

        output._backward = _backward
        return output



    #################
    # REDUCTIVE OPS #
    #################
    def sum(input,axis = None, keepdims = False):
        output = ag.Tensor(np.sum(input.value, axis = axis, keepdims = keepdims), inputs = [input], op='sum')
        def _backward():
            if axis == None or keepdims:
                input.grad += output.grad
            else:
                input.grad += np.expand_dims(output.grad, axis = axis)
            return None
        output._backward = _backward
        return output

    def matmul(input1, input2):
        return input1@input2
    ###############
    # SHAPING OPS #
    ###############

    def expand_dims(input, axis):
        output = ag.Tensor(np.expand_dims(input.value,axis=axis), inputs = [input])
        def _backward():
            input.grad += np.squeeze(output.grad, axis = axis)
            return None
        output._backward = _backward
        return output

    def reshape(input, newshape):
        output = ag.Tensor(np.reshape(input.value, newshape), inputs=[input], op="reshape")
        def _backward():
            input.grad += np.reshape(output.grad, input.shape)
            return None
        output._backward = _backward
        return output

    def moveaxis(input, source, destination):
        output = ag.Tensor(np.moveaxis(input.value, source, destination), inputs=[input], op="moveaxis")

        def _backward():
            input.grad += np.moveaxis(output.grad, source, destination)
            return None
        output._backward = _backward
        return output

    # [sp]arse [c]onstant (non-AG-enabled) [mat]rix [mul]tiplication
    def spcmatmul(input, sparse_mat: csr_matrix):
        output = ag.Tensor(input.value @ sparse_mat,
                           inputs = [input],
                           op="spcmatmul")

        def _backward():
            input.grad += output.grad @ sparse_mat.T
            return None

        output._backward = _backward
        return output

    class Tensor: # Tensor with grads
        def __init__(self,
                     value,
                     op="",
                     _backward= lambda : None,
                     inputs=[],
                     label="",
                     requires_grad=True):

            if type(value) in [float ,int]:
                value = np.array(value).astype(np.float64)
            self.value = value.astype(np.float64)
            self.grad = np.zeros_like(self.value).astype(np.float64)

            self.shape = value.shape

            self._backward = _backward
            self.inputs = inputs

            self.op = op
            self.label = label

            self.requires_grad = requires_grad

        def topological_sort(self):
            topo_order = []
            visited = set()

            def dfs(node):
                if node not in visited:
                    visited.add(node)
                    for input in node.inputs:
                        dfs(input)
                    topo_order.append(node)

            dfs(self)
            return topo_order

        def backward(self):
            self.grad = np.array(1.0).astype(np.float64)

            topo_order = self.topological_sort()

            for node in reversed(topo_order):
                node._backward()



        def __add__(self, other):

            if type(other) in [float, int]:
                other = ag.Tensor(1.0*other)
            result_shape, ax1, pad1, ax2, pad2 = broadcasted_shape(self.shape, other.shape)

            output = ag.Tensor(self.value + other.value,
                               inputs=[self, other], op="add")
            def _backward():
                self.grad += unbroadcast(output.grad, ax1, pad1)
                other.grad += unbroadcast(output.grad, ax2, pad2)

            output._backward = _backward
            return output

        def __sub__(self,other):
            return self + other*(-1)

        def __neg__(self):
            output = ag.Tensor(-self.value, inputs=[self], op="neg")
            def _backward():
                self.grad -= output.grad
                return None
            output._backward = _backward
            return output

        def __mul__(self, other):
            if type(other) in [float, int]:
                other = ag.Tensor(1.0*other)
            result_shape, ax1, pad1, ax2, pad2 = broadcasted_shape(self.shape, other.shape)

            output = ag.Tensor(self.value * other.value,
                               inputs=[self, other], op="mul")
            def _backward():
                self.grad += unbroadcast(output.grad*other.value, ax1, pad1)
                other.grad += unbroadcast(output.grad*self.value, ax2, pad2)

            output._backward = _backward
            return output

        def __truediv__(self,other):
            return self*(other**(-1))


        def __pow__(self, exponent): # exponent is just a python float
            output = ag.Tensor(self.value ** exponent,
                               inputs=[self],
                               op=f"pow({exponent})")

            def _backward():

                self.grad += (exponent * self.value**(exponent-1)) * output.grad
                return None

            output._backward = _backward
            return output


        def __getitem__(self, idx):
            output = ag.Tensor(np.array(self.value[idx]),
                               inputs = [self],
                               op=f"[...]")
            def _backward():
                self.grad[idx] += output.grad # idx must not have repeats!
                return None
            output._backward = _backward
            return output

        def __radd__(self, other):
            return self + other

        def __rmul__(self, other):
            return self * other

        def __rsub__(self, other):
            return (-self) + other

        def __rtruediv__(self, other):
            return ag.Tensor(other) / self


        def __matmul__(self,other):
            output = ag.Tensor(np.matmul(self.value,other.value),
                               inputs = [self,other],
                               op="matmul")

            # self.shape = (a,b,c, m,n)
            # other.shape = (a,b,c, n,p)
            # output.grad.shape == output.value.shape = (a,b,c, m,p)


            def _backward():
                if len(other.value.shape) == 1:
                    if len(self.value.shape) == 1:
                        raise Exception("To take the dot-product of two vectors, use ag.sum(x*y) instead.")
                    else:
                        self.grad += np.matmul(output.grad[:,None], other.value[None,:])
                else:
                    self.grad += np.matmul(output.grad, np.moveaxis(other.value,-1,-2))
                axis = list(range(len(self.value.shape)-1))
                other.grad += np.tensordot(self.value, output.grad, axes = [axis,axis])
                return None
            output._backward = _backward
            return output

        def reshape(self, *newshape):
            output = ag.Tensor(np.reshape(self.value, tuple(newshape)), inputs=[self], op="reshape")
            def _backward():
                self.grad += np.reshape(output.grad, self.shape)
                return None
            output._backward = _backward
            return output



        def __repr__(self) -> str:
            return "Value:\n"+self.value.__repr__() + "\nGrad:\n" + self.grad.__repr__()
        # def __repr__(self) -> str:
        #     return "Value:\n"+self.value.__repr__() + "\nGrad:\n" + self.grad.__repr__()

In [ ]:
class nn:
    # A fake python class so that we can use nn as an "namespace"
    # This way we can avoid having an actual python module
    # which would create more files for you to have to download
    # and for me to upload.

    # But the sacrifice is that the class inheritance is a bit wonky
    # Maybe someone more well-versed in python class inheritance can help me out
    # Email me if you'd like to contribute to making this class better next time

    class Module:
        def __init__(self):
            self._parameters = {}
            self._modules = {}

        def forward(self, *inputs):
            raise NotImplementedError

        def __call__(self, *inputs):
            return self.forward(*inputs)

        def parameters(self):
            params = list(self._parameters.values())
            for module in self._modules.values():
                params.extend(module.parameters())
            return params

        def __setattr__(self, name, value):
            object.__setattr__(self, name, value)
            if isinstance(value, nn.Module):
                self._modules[name] = value
            elif isinstance(value, ag.Tensor):
                self._parameters[name] = value


    class Linear(Module):
        def __init__(self, in_features, out_features):
            nn.Module.__init__(self)
            kaiming_he_init_constant = np.sqrt(2 / in_features)

            self.weight = ag.Tensor(np.random.randn(in_features, out_features) * kaiming_he_init_constant )
            self.bias = ag.Tensor(np.zeros(out_features))

            # these do not trigger __setattr__
            self._parameters['weight'] = self.weight
            self._parameters['bias'] = self.bias

        def forward(self, x):
            return ag.matmul(x, self.weight) + self.bias

    class AvgPool2d(Module):
        def __init__(self, kernel_size):
            super().__init__()
            self.kernel_size = kernel_size
            self.im2col_mat = None

        def forward(self, Xin):
            N, Cin, Hin, Win = Xin.shape
            assert(Hin % self.kernel_size == 0)
            assert(Win % self.kernel_size == 0)

            K = self.kernel_size
            S = self.kernel_size

            # Yang Xu's more slick implementation
            Hout = Hin//S
            Wout = Win//S

            Xin_patches = Xin.reshape(N, Cin, Hout, K, Wout, K)
            Xout = ag.sum(Xin_patches, axis=(-1, -3))/K**2

            # Hout = (Hin - K) // S + 1
            # Wout = (Win - K) // S + 1

            # P = Hout * Wout  # Total number of patches per image
            # patch_size = Cin * K * K

            # Xin_flat = Xin.reshape(-1, Cin * Hin * Win)

            # if self.im2col_mat is None:
            #     self.im2col_mat = im2col_matrix_sparse(Xin, K, S)

            # Xin_im2col = ag.spcmatmul(Xin_flat, self.im2col_mat)
            # Xin_patches_flat = Xin_im2col.reshape(N, P, patch_size)

            # Xin_cw_patches_flat = Xin_patches_flat.reshape(N, P, Cin, K**2)
            # Xout = ag.moveaxis(ag.sum(Xin_cw_patches_flat, axis=-1), -1, -2).reshape(N, Cin, Hout, Wout) / K**2

            return Xout

    class Conv2d(Module):
        def __init__(self, in_channels, out_channels, kernel_size, stride=1):
            super().__init__()
            self.in_channels = in_channels # Cin
            self.out_channels = out_channels # Cout
            self.kernel_size = kernel_size # K
            self.stride = stride # S

            kaiming_he_init_constant = np.sqrt(2 / (in_channels * kernel_size**2))

            # these are flattened weights
            self.weight = ag.Tensor(np.random.randn(in_channels*kernel_size**2, out_channels) * kaiming_he_init_constant)

            # bias are set to zeros initially
            self.bias = ag.Tensor(np.zeros(out_channels))

            self._parameters['weight'] = self.weight
            self._parameters['bias'] = self.bias

            self.im2col_mat = None # im2col_mat will be cached

        def forward(self, Xin):
            # Xin: (N, Cin, Hin, Win)
            N, Cin, Hin, Win = Xin.shape
            assert(Cin == self.in_channels)

            K = self.kernel_size
            S = self.stride
            Hout = (Hin - K) // S + 1
            Wout = (Win - K) // S + 1
            P = Hout * Wout  # Total number of patches per image
            patch_size = Cin * K * K  # Size of each flattened patch
            Cout = self.out_channels

            Xin_flat = Xin.reshape(-1, Cin * Hin * Win)

            # Cache the im2col matrix
            if self.im2col_mat is None:
                self.im2col_mat = im2col_matrix_sparse(Xin, K, S)

            Xin_im2col = ag.spcmatmul(Xin_flat, self.im2col_mat)
            Xin_patches_flat = Xin_im2col.reshape(N, P, patch_size)


            Xout_flat = (Xin_patches_flat @ self.weight) + self.bias
            Xout_flat = ag.moveaxis(Xout_flat, 1, 2)
            Xout = Xout_flat.reshape(N, Cout, Hout, Wout)

            # Xout: (N, Cout, Hout, Wout)
            return Xout


    class MSELoss:
        def __call__(self, input, target):
            N = target.value.shape[0]
            return ag.sum((target - input)**2) / N

    class BinaryCrossEntropyLoss:
        def __call__(self, input, target):
            N = target.value.shape[0]
            return ag.sum( ag.log(1.0+ ag.exp(-input*target))) / N

    class CrossEntropyLoss:
        def __call__(self, input, target):
            # input should be an ag.Tensor of shape (N, num_classes)
            # target should be a numpy array of shape (N,) of elements of [0,1,..., num_classes-1]

            z = input
            y = target

            expz = ag.exp(z)

            p = expz / ag.sum(expz, axis=-1, keepdims=True)
            N = z.shape[0]
            l = ag.sum(-ag.log(p[np.arange(N), y]))/N
            return l

In [ ]:
class optim:
    class SGD:
        def __init__(self, parameters, lr=0.01):
            self.parameters = parameters
            self.lr = lr

        def step(self):
            for p in self.parameters:
                p.value -= self.lr * p.grad

        def zero_grad(self):
            for p in self.parameters:
                p.grad = np.zeros_like(p.value)

In [ ]:
loaded_images = (1.0/255)*np.loadtxt('lec09_images.csv', delimiter=",").reshape(-1,1,28,28)

# padding
loaded_images = np.pad(loaded_images, ((0,0), (0,0), (2,2), (2,2))) # ensure 32-by-32, as in LeCun et al 1998
# this is a preprocess step, no need to track this op through out autograd
# although later on, it might be a good idea to add "pad" as an op

X = ag.Tensor(1.0*loaded_images)
y = np.repeat(np.arange(10), 8)

In [ ]:
fig, axes = plt.subplots(2, 5, figsize=(6, 3))

for i in range(10):
    ax = axes[i // 5, i % 5]
    ax.imshow(loaded_images[i * 8][0], cmap='gray')
    ax.set_title(y[i*8])
    ax.axis('off')

plt.tight_layout()
plt.show()

# Testing your convolution layer

In [ ]:
conv_toy_example = nn.Conv2d(in_channels=1,out_channels=2, kernel_size=2)


conv_toyexweight = np.array([[ 0.,  -1.],
                             [ 1.,  0.],
                             [-1.,  1.],
                             [ 0.,  0.]])
conv_toy_example.weight = ag.Tensor(conv_toyexweight)

X_conved = conv_toy_example(X)

# Channel 1

In [ ]:

fig, axes = plt.subplots(2, 5, figsize=(6, 3))

for i in range(10):
    ax = axes[i // 5, i % 5]
    ax.imshow(X_conved.value[i * 8][0], cmap='gray')
    ax.set_title(y[i*8])
    ax.axis('off')

plt.tight_layout()
plt.show()

# Channel 2

In [ ]:
fig, axes = plt.subplots(2, 5, figsize=(6, 3))

for i in range(10):
    ax = axes[i // 5, i % 5]
    ax.imshow(X_conved.value[i * 8][1], cmap='gray')
    ax.set_title(y[i*8])
    ax.axis('off')

plt.tight_layout()
plt.show()

# Testing your AvgPool layer

In [ ]:
avgpool_test = nn.AvgPool2d(kernel_size=2)


X_pooled = avgpool_test(X)

fig, axes = plt.subplots(2, 5, figsize=(6, 3))

for i in range(10):
    ax = axes[i // 5, i % 5]
    ax.imshow(X_pooled.value[i * 8][0], cmap='gray')
    ax.set_title(y[i*8])
    ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
class LeNet5(nn.Module):
    def __init__(self):
        nn.Module.__init__(self)

        # Convolutional layers (Conv2d + AvgPool2d)
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.avgpool1 = nn.AvgPool2d(2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.avgpool2 = nn.AvgPool2d(2)

        # Fully-connected layers
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Convolutional (conv) component of the network
        x = self.conv1(x)
        x = ag.relu(x)
        x = self.avgpool1(x)

        x = self.conv2(x)
        x = ag.relu(x)
        x = self.avgpool2(x)

        # Flatten the output from conv layers to feed into fully connected layers
        x = x.reshape(x.shape[0], -1)

        # Fully-connected (fc) component of the network
        x = self.fc1(x)
        x = ag.relu(x)

        x = self.fc2(x)
        x = ag.relu(x)

        x = self.fc3(x)

        return x


In [ ]:
model = LeNet5()
optimizer = optim.SGD(model.parameters(), lr=0.1)
criterion = nn.CrossEntropyLoss()

# How good is the model initially?

In [ ]:
np.argmax(model(X).value,axis=-1) == y

# Count the number of parameters

In [ ]:
np.sum([np.prod(param.value.shape) for param in list(model.parameters())])

In [ ]:


num_epochs = 100
start_time = time.time()

for epoch in range(num_epochs):



    optimizer.zero_grad()
    outputs = model(X)
    loss = criterion(outputs, y)
    loss.backward()
    optimizer.step()


    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.value:.10f}')
end_time = time.time()
total_time = end_time - start_time

print(f"Total training time: {total_time:.4f} seconds")


# It should achieve 100 percent training acc

In [ ]:
np.argmax(model(X).value,axis=-1) == y